#### Import Libs

In [229]:
import os
import re
import numpy as np
import simpy
import time
from numpy import random

<hr>

1)

####  File Utils

In [2]:
def read_file(folder, filename):
    filepath = os.path.join(folder, filename)
    file = open(filepath, 'r')
    page = file.read()
    return page

In [3]:
link_regex = '<a href=\"http://(.*?).html'

#### Page Rank

In [4]:
def open_pages(folder):
    filenames = os.listdir(folder)
    page_names = list(map(lambda filename: filename.split('.')[0], filenames))
    pages = []
    for filename in filenames:        
        page = read_file(folder, filename)
        pages.append(page)
    return (pages, page_names)

def markov_row(page, page_names):
    # n pages
    n = len(page_names)
    
    # search links in the page
    matchs = re.findall(link_regex, page)
    
    # build adjacency
    row = []
    if len(matchs) == 0:
        row = [1/n, 1/n, 1/n, 1/n, 1/n, 1/n]
    else:
        for page_name in page_names:
            n_matchs = 0
            for match in matchs:
                if page_name == match:
                    n_matchs += 1
            row.append(n_matchs / len(matchs))
    
    return row

def markov_matrix(pages, page_names):
    matrix = np.zeros(shape=(len(page_names), len(page_names)))
    for index, page in enumerate(pages):
        row = markov_row(page, page_names)
        matrix[index] = row
    return matrix

def distribucion_estacionaria(A):
    A = [1,0,0,0,0,0]
    for i in range(1000):
        A = np.dot(A,matrix)
    return A

#### Run

In [16]:
folder = 'paginas'
pages, page_names = open_pages(folder)
matrix = markov_matrix(pages, page_names)

print("Markov Matrix\n")
print(matrix)
print("\n")

page_rank = distribucion_estacionaria(matrix)
print("Page Rank\n")
aux = dict()

for i in range(len(page_names)):
    aux[page_names[i]]=page_rank[i]
    print(page_names[i][:13],"\t",page_rank[i])
page_rank = aux

# Sort key
def value_getter(item):
     return item[1]

sorted(aux.items(), key=value_getter,reverse=True)

Markov Matrix

[[0.         0.5        0.         0.         0.         0.5       ]
 [0.5        0.         0.25       0.         0.25       0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.         0.         1.         0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.66666667 0.33333333 0.         0.         0.         0.        ]]


Page Rank

angelinajolie 	 0.28571428571428525
bradpitt 	 0.24489795918367308
martinscorces 	 0.14285714285714263
robertdeniro 	 0.04081632653061218
jenniferanist 	 0.10204081632653045
jonvoight 	 0.1836734693877548


[('angelinajolie', 0.28571428571428525),
 ('bradpitt', 0.24489795918367308),
 ('jonvoight', 0.1836734693877548),
 ('martinscorcese', 0.14285714285714263),
 ('jenniferaniston', 0.10204081632653045),
 ('robertdeniro', 0.04081632653061218)]

In [40]:
import string
folder = 'paginas'
filenames = os.listdir(folder)

indexes=dict()
for filename in filenames: 
    
    filepath = os.path.join(folder, filename)

    text = open(filepath, 'r')

    # Create an empty dictionary
    d = dict()
    # Loop through each line of the file
    for line in text:
        # Remove the leading spaces and newline character
        line = line.strip()

        # Convert the characters in line to
        # lowercase to avoid case mismatch
        line = line.lower()

        # Remove the punctuation marks from the line
        line = line.translate(line.maketrans("", "", string.punctuation))

        # Split the line into words
        words = line.split(" ")

        # Iterate over each word in line
        for word in words:
            # Check if the word is already in dictionary
            if word in d:
                # Increment count of word by 1
                d[word] = d[word] + 1
            else:
                # Add the word to dictionary with count 1
                d[word] = 1

                
    indexes[filename.split('.')[0]] = d

def calculate_scores(word):
    scores=dict()
    for key in list(indexes.keys()):
        if indexes[key].get(word) is None:
            # si no encuentra la palabra directamente le pone puntaje 0
            scores[key] = 0
        else:
            # ponderacion basica entre 
            scores[key]=0.5* indexes[key].get(word) + 0.5*page_rank[key]
    return sorted(scores.items(), key=lambda item: item[1],reverse=True)

scores=calculate_scores("film")
print(scores)

scores=calculate_scores("action")
print(scores)

scores=calculate_scores("actor")
print(scores)

[('martinscorcese', 3.071428571428571), ('angelinajolie', 1.6428571428571426), ('bradpitt', 1.6224489795918364), ('jonvoight', 0.5918367346938774), ('jenniferaniston', 0.5510204081632653), ('robertdeniro', 0.5204081632653061)]
[('angelinajolie', 0), ('bradpitt', 0), ('martinscorcese', 0), ('robertdeniro', 0), ('jenniferaniston', 0), ('jonvoight', 0)]
[('jonvoight', 2.0918367346938775), ('robertdeniro', 1.5204081632653061), ('bradpitt', 1.1224489795918364), ('martinscorcese', 1.0714285714285714), ('angelinajolie', 0.6428571428571426), ('jenniferaniston', 0)]


<hr>

3)

<b>Web Service</b>. El cual consulta una <b>Base de Datos</b>.

Dos opciones: <br>
a) Utilizar dos bases de datos => <b>M/M/2</b> (dos canales de atencion) <br>
b) Utilizar una base de datos => <b>M/M/1</b> (un canal de atencion) <br>

Para ambos casos el tiempo entre arribos es 4 segundos:

In [211]:
# 1 arribo  ______ 4 segundos
# x arribos ______ 1 segundo
#
# x = (1 segundo * 1 arribo) / 4 segundos = 1/4 arribos en un segundo

$$
\lambda = \frac{1}{4}
$$

b)

Tiempo de servicio es 0.8 segundos, esta es la tasa de servicio.

$$
\mu = 0.8
$$

In [287]:
class Request:
    def __init__(self, id, timestamp):
        self.id = id
        self.timestamp = timestamp

class WebService:
    n = 10
    
    request_id = 0
    requests = []
    
    media_request = 0.25
    media_procesamiento = 0.8
    
    n_requests = 0
    n_procesadas = 0
    
    n_request_sin_procesar = 0
    
    total_elapsed = 0

    def __init__(self, env):
        self.env = env
        
    def get_media(self):
        return self.total_elapsed / self.n_procesadas
        
    def generador_requests(self):
        while self.n_procesadas < self.n:
            
            self.n_requests += 1
            self.n_request_sin_procesar += 1
            
            request = Request(self.n_requests, self.env.now)
            self.requests.append(request)
            
            print("proceso de requests | media: {} | n requess: {} | requests sin procesar: {}".format(self.media_request, self.n_requests, self.n_request_sin_procesar))
            
            tiempo = random.exponential(self.media_request)
            yield self.env.timeout(tiempo)
    
    def generador_procesamiento(self):
        while self.n_procesadas < self.n:
            if self.n_request_sin_procesar > 0:
       
                request = self.requests.pop()
                elapsed = self.env.now - request.timestamp
                print("elapsed: {}".format(elapsed))
                self.total_elapsed += elapsed
                
                self.n_procesadas += 1
                self.n_request_sin_procesar -= 1
                
                print("proceso de procesamiento | media: {} | n procesadas: {} | requests sin procesar: {}".format(self.media_procesamiento, self.n_procesadas, self.n_request_sin_procesar))
                
            tiempo = random.exponential(self.media_procesamiento)
            yield self.env.timeout(tiempo)

In [288]:
env = simpy.Environment()
web_service = WebService(env)
env.process(web_service.generador_requests())
env.process(web_service.generador_procesamiento())
env.run()

proceso de requests | media: 0.25 | n requess: 1 | requests sin procesar: 1
elapsed: 0
proceso de procesamiento | media: 0.8 | n procesadas: 1 | requests sin procesar: 0
proceso de requests | media: 0.25 | n requess: 2 | requests sin procesar: 1
proceso de requests | media: 0.25 | n requess: 3 | requests sin procesar: 2
elapsed: 0.13166544866232677
proceso de procesamiento | media: 0.8 | n procesadas: 2 | requests sin procesar: 1
proceso de requests | media: 0.25 | n requess: 4 | requests sin procesar: 2
proceso de requests | media: 0.25 | n requess: 5 | requests sin procesar: 3
elapsed: 0.030108306323837652
proceso de procesamiento | media: 0.8 | n procesadas: 3 | requests sin procesar: 2
proceso de requests | media: 0.25 | n requess: 6 | requests sin procesar: 3
proceso de requests | media: 0.25 | n requess: 7 | requests sin procesar: 4
elapsed: 0.014512309381548683
proceso de procesamiento | media: 0.8 | n procesadas: 4 | requests sin procesar: 3
proceso de requests | media: 0.25 | 

Tiempo medio de espera entre que la solicitud llega y puede ser procesada.

In [289]:
web_service.get_media()

0.17575604709072565